#Clinical Trials LLM

**Install necessary libraries**

In [1]:
!pip install openai --upgrade
!pip install langchain_openai --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.3/377.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 3.5 MB/s eta 0:00:00


### Import and Pre-Process Clinical Trial Data from clinicaltrials.gov

In [2]:
import requests
import pandas as pd

# Initial URL for the first API call
base_url = "https://clinicaltrials.gov/api/v2/studies"
params = {
    "query.titles": "Breast Cancer",
    "pageSize": 100
}

# Initialize an empty list to store the data
data_list = []

# Loop until there is no nextPageToken
while True:
    # Print the current URL (for debugging purposes)
    print("Fetching data from:", base_url + '?' + '&'.join([f"{k}={v}" for k, v in params.items()]))

    # Send a GET request to the API
    response = requests.get(base_url, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()  # Parse JSON response
        studies = data.get('studies', [])  # Extract the list of studies

        # Loop through each study and extract specific information
        for study in studies:
            # Safely access nested keys
            nctId = study['protocolSection']['identificationModule'].get('nctId', 'Unknown')
            overallStatus = study['protocolSection']['statusModule'].get('overallStatus', 'Unknown')
            startDate = study['protocolSection']['statusModule'].get('startDateStruct', {}).get('date', 'Unknown Date')
            conditions = ', '.join(study['protocolSection']['conditionsModule'].get('conditions', ['No conditions listed']))
            acronym = study['protocolSection']['identificationModule'].get('acronym', 'Unknown')

            # Extract interventions safely
            interventions_list = study['protocolSection'].get('armsInterventionsModule', {}).get('interventions', [])
            interventions = ', '.join([intervention.get('name', 'No intervention name listed') for intervention in interventions_list]) if interventions_list else "No interventions listed"

            # Extract locations safely
            locations_list = study['protocolSection'].get('contactsLocationsModule', {}).get('locations', [])
            locations = ', '.join([f"{location.get('city', 'No City')} - {location.get('country', 'No Country')}" for location in locations_list]) if locations_list else "No locations listed"

            # Extract dates and phases
            primaryCompletionDate = study['protocolSection']['statusModule'].get('primaryCompletionDateStruct', {}).get('date', 'Unknown Date')
            studyFirstPostDate = study['protocolSection']['statusModule'].get('studyFirstPostDateStruct', {}).get('date', 'Unknown Date')
            lastUpdatePostDate = study['protocolSection']['statusModule'].get('lastUpdatePostDateStruct', {}).get('date', 'Unknown Date')
            studyType = study['protocolSection']['designModule'].get('studyType', 'Unknown')
            phases = ', '.join(study['protocolSection']['designModule'].get('phases', ['Not Available']))

            # Extract eligibility criteria
            #eligibilities = ', '.join(study['protocolSection']['eligibilityModule'].get('eligibilityCriteria', ['No criteria listed']))

            eligibility = study['protocolSection']['eligibilityModule'].get('eligibilityCriteria', 'Unknown')

            #eligibilities_list = study['protocolSection'].get('eligibilityModule', {}).get('eligibilityCriteria', [])
            #eligibilities = ', '.join([eligiblity.get('name', 'No criteria listed') for eligiblity in eligibilities_list]) if eligibilities_list else "No eligibility listed"

            # Append the data to the list as a dictionary
            data_list.append({
                "NCT ID": nctId,
                "Acronym": acronym,
                "Overall Status": overallStatus,
                "Start Date": startDate,
                "Conditions": conditions,
                "Interventions": interventions,
                "Locations": locations,
                "Primary Completion Date": primaryCompletionDate,
                "Study First Post Date": studyFirstPostDate,
                "Last Update Post Date": lastUpdatePostDate,
                "Study Type": studyType,
                "Phases": phases,
                "Eligibility": eligibility
            })

        # Check for nextPageToken and update the params or break the loop
        nextPageToken = data.get('nextPageToken')
        if nextPageToken:
            params['pageToken'] = nextPageToken  # Set the pageToken for the next request
        else:
            break  # Exit the loop if no nextPageToken is present
    else:
        print("Failed to fetch data. Status code:", response.status_code)
        break

# Create a DataFrame from the list of dictionaries
clinical_trial_data = pd.DataFrame(data_list)

# Print the DataFrame
#print(clinical_trial_df)

# Optionally, save the DataFrame to a CSV file
clinical_trial_data .to_csv("clinical_trials_data.csv", index=False)

Fetching data from: https://clinicaltrials.gov/api/v2/studies?query.titles=Breast Cancer&pageSize=100
Fetching data from: https://clinicaltrials.gov/api/v2/studies?query.titles=Breast Cancer&pageSize=100&pageToken=NF0g5JCFkvgp
Fetching data from: https://clinicaltrials.gov/api/v2/studies?query.titles=Breast Cancer&pageSize=100&pageToken=NF0g5JSEkPAh
Fetching data from: https://clinicaltrials.gov/api/v2/studies?query.titles=Breast Cancer&pageSize=100&pageToken=NF0g5JKHlfArxw
Fetching data from: https://clinicaltrials.gov/api/v2/studies?query.titles=Breast Cancer&pageSize=100&pageToken=NF0g5JKDkfEoxQ
Fetching data from: https://clinicaltrials.gov/api/v2/studies?query.titles=Breast Cancer&pageSize=100&pageToken=NF0g5JKBmvYvyQ
Fetching data from: https://clinicaltrials.gov/api/v2/studies?query.titles=Breast Cancer&pageSize=100&pageToken=NF0g5JGHk_Uoxg
Fetching data from: https://clinicaltrials.gov/api/v2/studies?query.titles=Breast Cancer&pageSize=100&pageToken=NF0g5JGFm_gtwg
Fetching data

In [3]:
clinical_trial_data = clinical_trial_data[clinical_trial_data['Overall Status'] == 'RECRUITING']

In [6]:
print('Number of Clinical Trials: ' + str(len(clinical_trial_data)))

Number of Clinical Trials: 1764


### Import Patient Records

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
#!ls /content/drive/MyDrive/synthea_dataset

'Adrian111_Bartell116_fd25b51b-dc2a-4edb-b7c9-2c5d178a0ee7 (1).json'
 Adrian111_Bartell116_fd25b51b-dc2a-4edb-b7c9-2c5d178a0ee7.json
 Adrianna470_Boyer713_6fcf56ed-8ad8-4395-a966-9ebee3822656.json
 Adrienne302_Zulauf375_1c83fb03-d139-4626-bb39-4a062c22b533.json
 Alaine226_Kassulke119_5efa5654-8190-4f4d-9ba3-3b79448440b5.json
 Ali918_Larson43_896d3035-d036-4c8a-9901-367dc8814495.json
 Allegra202_Heller342_cc4c634f-c2ff-4963-b75a-fa2e33dc208f.json
 Allena79_Pacocha935_d11e7fb0-5853-4762-b8fa-6cd9008a790b.json
 Almeda560_Davis923_c762bbaa-b207-4b4d-8123-e5bc93e3b99b.json
 Ana_María762_Gamez720_de3666a6-b4e3-4ec8-9ae7-bde168e03465.json
 Anne636_Collins926_42c179ad-99bd-4ca4-861f-7151d774a243.json
 Annelle169_King743_ddd8d7a4-5992-4572-a231-80bc305298b1.json
 Annemarie794_Kihn564_907083ad-22c9-4067-8321-a6622ce9792c.json
 Aracely711_Carter549_31665402-bfc5-45e1-b12a-fdc42836f50a.json
 Arden380_Reichel38_9c6e8766-9da5-428e-be61-e81a78a920c2.json
 Arlean815_MacGyver246_f9e2ec3e-731f-4341-8a6

In [ ]:
def flatten_dataframe(df):
    for column in df.columns:
        if df[column].apply(lambda x: isinstance(x, (list, dict))).any():
            # Normalize the nested column
            normalized_df = pd.json_normalize(df[column].explode().dropna().tolist(), sep='_')
            normalized_df.columns = [f"{column}_{sub_col}" for sub_col in normalized_df.columns]
            # Drop the original nested column and join the normalized columns
            df = df.drop(columns=[column]).join(normalized_df)
    return df

In [ ]:
patient_files = []
patient_data = {}
patient_names = []

In [ ]:
import json
from collections import defaultdict
import pandas as pd

for patient_file in patient_files:
  patient_name = patient_file.split('/')[-1].split('.')[0]
  patient_name = '_'.join(patient_name.split('_')[:2])
  print(patient_name)
  patient_names.append(patient_name)

  patient = {}
  #Load patient file
  with open(patient_file) as f:
    data = json.load(f)

  # Extract the entries
  entries = data.get('entry', [])

  # Group by resourceType
  resource_dict = {}
  for entry in entries:
    resource = entry.get('resource', {})
    resource_type = resource.get('resourceType', 'Unknown')
    if resource_type not in resource_dict:
        resource_dict[resource_type] = []
    resource_dict[resource_type].append(resource)

  for resource_type, resources in resource_dict.items():
    # Normalize the entire resource dictionary and separate nested tags
    df = pd.json_normalize(resources, sep='_')

    # Flatten the nested columns
    df = flatten_dataframe(df)

    # Filter out inactive patient information
    #if 'status' in df.columns:
      #print(df['status'].unique())
        #df = df[(df['status'] == 'active') | (df['status'] == 'current') | (df['status'] == 'final') | (df['status'] == 'finished') | (df['status'] == 'completed')]

    if 'id' in df.columns:
        df = df.drop(columns=['id'])

    columns_to_drop = [col for col in df.columns if any(substring in col for substring in ['reference', 'system', 'telecom', 'identifier', 'Identifier', 'uid', 'Address', 'address'])]
    df = df.drop(columns=columns_to_drop)

    df = flatten_dataframe(df) #handle second layer of nesting

    patient[resource_type] = df

  patient_data[patient_name] = patient

### **Part 1:** Query LLM with Full Patient Record and Full Clinical Trial Data

### **Part 2:** Prompt LLM to Create a Patient Summary from the Full Patient Record

### **Part 3:** Query LLM with Patient Summary and Full Clinical Trial Data

In [ ]:
llm_responses = {}

In [ ]:
import os
from langchain_openai import ChatOpenAI

os.environ["OPENAI_API_KEY"] = 'sk-proj-bbcCqgJsIPOhjmiRfNdyT3BlbkFJuDf9IePweh97i2lYZkTl' # API Key from Jieming

#model
model = "gpt-4-turbo" # most powerful gpt model
llm = ChatOpenAI(temperature=0.1, model=model)

In [ ]:
from datetime import date
today_date = date.today()

for patient in patient_names:
  print(patient)

  responses = {}

  # Trial 1: Query LLM with Full Patient Data and Full Clinical Trial Data
  patient_df = patient_data[patient]

  query1 =

  response1 = llm.invoke(query)

  responses['Full+Full'] = response1.content

  # Trial 2: Query LLM to Write a Patient Summary
  query2 =

  response2 = llm.invoke(query)

  responses['Patient_Summary'] = response2.content

  # Trial 3: Query LLM with Patient Summary and and Full Clinical Trial Data
  query3 =

  response3 = llm.invoke(query)

  responses['Summary+Full'] = response3.content

  llm_responses[patient] = responses

### **Part 4:** Query RAG-Enabled LLM with Patient Summary and Text-Based Clinical Trial Data

In [ ]:
from openai import OpenAI

client = OpenAI(
  api_key=os.environ["OPENAI_API_KEY"],  # this is also the default, it can be omitted
)

clinical_trial_data['text'] = 'The trial with NCT ID number ' + clinical_trial_data['NCT ID'] + ' has the following eligbility criteria, for inclusion and exclusion ' + clinical_trial_data['Eligibility']
clinical_trial_data.head()['text']

In [ ]:
import openai
import tiktoken
from scipy import spatial
import pandas as pd
import ast
import openai

# Function to get text embeddings
def text_embedding(text):
    response = client.embeddings.create(model="text-embedding-ada-002", input=text)
    return response.data[0].embedding

# Apply the text_embedding function to the 'text' column
clinical_trial_data['embedding'] = clinical_trial_data['text'].apply(lambda x: text_embedding(x))

clinical_trial_data = clinical_trial_data.assign(embedding = (clinical_trial_data["text"].apply(lambda x : text_embedding(x))))
#print(clinical_trial_data.head())

In [ ]:
def strings_ranked_by_relatedness(
    query: str,
    df: pd.DataFrame,
    relatedness_fn = lambda x, y: 1 - spatial.distance.cosine(x, y),
    top_n: int = 100
):

    EMBEDDING_MODEL = "text-embedding-ada-002"
    query_embedding_response = openai.embeddings.create(
        model = EMBEDDING_MODEL,
        input = query,
    )
    query_embedding = query_embedding_response.data[0].embedding
    strings_and_relatednesses = [
        (row["text"], relatedness_fn(query_embedding, row["embedding"]))
        for i, row in df.iterrows()
    ]
    strings_and_relatednesses.sort(key = lambda x: x[1], reverse = True)
    strings, relatednesses = zip(*strings_and_relatednesses)
    return strings[:top_n], relatednesses[:top_n]

strings, relatednesses = strings_ranked_by_relatedness("Clinical Trials", df, top_n = 3)
for string, relatedness in zip(strings, relatednesses):
    print(f"{relatedness = :.3f}")
    display(string)

def num_tokens(text: str) -> int:
    encoding = tiktoken.encoding_for_model("gpt-4-turbo")
    return len(encoding.encode(text))

def query_message(
    query: str,
    df: pd.DataFrame,
    model: str,
    token_budget: int
) -> str:
    strings, relatednesses = strings_ranked_by_relatedness(query, df)
    introduction = 'Use the below content related to clinical trials to answer the subsequent question. If the answer cannot be found in the articles, write "I could not find an answer."'
    question = f"\n\nQuestion: {query}"
    message = introduction
    for string in strings:
        next_row = f'\n\nClinical trial section:\n"""\n{string}\n"""'
        if (
            num_tokens(message + next_row + question)
            > token_budget
        ):
            break
        else:
            message += next_row
    return message + question

def ask(
    query: str,
    df: pd.DataFrame = clinical_trial_data,
    model: str = "gpt-4-turbo",
    token_budget: int = 25000,
    print_message: bool = False,
) -> str:
    message = query_message(query, df, model=model, token_budget=token_budget)
    if print_message:
        print(message)
    messages = [
        {"role": "system", "content": "You match patients to clinical trials that they are eligible for."},
        {"role": "user", "content": message},
    ]
    response = openai.chat.completions.create(
        model = model,
        messages = messages,
        temperature = 0.1
    )
    response_message = response.choices[0].message.content
    return response_message

In [ ]:
for patient in patient_names:

  patient_summary = llm_responses[patient]['Patient_Summary']

  # Trial 4: Query LLM to Write a Patient Summary
  query4 = "The following is a patient diagnosed with a type of breast cancer: " + str(patient_summary) + "Identify up to 2 clinical trials that the patient is eligible for using the eligibility criteria and explain with four reasons, why the patient is eligible for the trial, and show eligibility criteria (inclusion and exclusion) for each trial."

  response4 = ask(query4)

  llm_responses[patient]['Summary+RAG'] = response4.content

### **Save Files**

In [ ]:
llm_responses_df = pd.DataFrame.from_dict(llm_responses, orient='index')
llm_responses_df.to_csv('llm_responses.csv')

In [ ]:
query = f"""You are a doctor seeing a patient diagnosed with a type of breast cancer with the following health record. {df}. Identify at most 10 clinical trials that this patient is eligible for based on the eligibility criteria from the following data: {}'
"""

In [ ]:


query = f"""You are a doctor seeing a patient diagnosed with a type of breast cancer with the following health record. Write a summary for this patient's cancer prognosis. {patient}'
"""

#print(query)

response = llm.invoke(query)

AIMessage(content="Mrs. Allegra202 Heller342, born on April 23, 1928, is a 96-year-old female patient diagnosed with malignant neoplasm of the breast (breast cancer). Her condition is currently active, and she has been under continuous care and monitoring for her condition.\n\n**Cancer Diagnosis and Treatment:**\n- **Diagnosis:** Malignant neoplasm of the breast (disorder)\n- **Clinical Status:** Active\n- **Medications:**\n  - **Tamoxifen citrate 10 MG Oral Tablet:** Used to treat breast cancer that has spread to other parts of the body (metastatic breast cancer), to treat breast cancer in certain patients after surgery and radiation therapy, and to reduce the chances of breast cancer in high-risk patients.\n  - **Verzenio 100 MG Oral Tablet:** Used in the treatment of HR-positive, HER2-negative breast cancer that has progressed after treatment with hormone therapy.\n\n**Care Management:**\n- Mrs. Heller has been part of several care plans aimed at managing her condition and improving

In [ ]:
query = f"""You are a doctor seeing a patient diagnosed with a type of breast cancer with the following health record. Write a summary for this patient's cancer prognosis. Include the patient's age as of today's date {today_date}. Include the patient's comorbidities, treatments, and cancer staging information. {patient}'
"""

#print(query)

llm.invoke(query)


AIMessage(content="Mrs. Allegra202 Heller342, born on April 23, 1928, is a 96-year-old female patient diagnosed with breast cancer. As of today, July 29, 2024, she has been under the care of healthcare providers at Morton Hospital and PCP187044, with Dr. Sharilyn202 Simonis280 and Dr. Onita767 Osinski784 as her primary care physicians.\n\n### Comorbidities:\nMrs. Heller has a history of several comorbid conditions including:\n- Essential hypertension (active)\n- Hyperlipidemia (active)\n- Osteoporosis (active)\n- History of myocardial infarction (active)\n- Coronary heart disease (active)\n\n### Cancer Details:\n- Type: Malignant neoplasm of breast (active)\n- Clinical Stage: Stage 2A (qualifier value) as per the latest staging assessment.\n\n### Treatments:\nMrs. Heller has been prescribed various medications for her comorbid conditions and cancer treatment:\n- Tamoxifen citrate 10 MG Oral Tablet\n- Verzenio 100 MG Oral Tablet\n- Aspirin 81 MG Oral Tablet\n- Simvastatin 20 MG Oral Tab

In [ ]:
patient_summary = "Mrs. Allegra202 Heller342, born on April 23, 1928, is a 96-year-old female patient diagnosed with breast cancer. As of today, July 29, 2024, she has been under the care of healthcare providers at Morton Hospital and PCP187044, with Dr. Sharilyn202 Simonis280 and Dr. Onita767 Osinski784 as her primary care physicians.\n\n### Comorbidities:\nMrs. Heller has a history of several comorbid conditions including:\n- Essential hypertension (active)\n- Hyperlipidemia (active)\n- Osteoporosis (active)\n- History of myocardial infarction (active)\n- Coronary heart disease (active)\n\n### Cancer Details:\n- Type: Malignant neoplasm of breast (active)\n- Clinical Stage: Stage 2A (qualifier value) as per the latest staging assessment.\n\n### Treatments:\nMrs. Heller has been prescribed various medications for her comorbid conditions and cancer treatment:\n- Tamoxifen citrate 10 MG Oral Tablet\n- Verzenio 100 MG Oral Tablet\n- Aspirin 81 MG Oral Tablet\n- Simvastatin 20 MG Oral Tablet\n- Hydrochlorothiazide 25 MG Oral Tablet\n- Nitroglycerin 400 MCG/ACTUAT Mucosal Spray\n- AmLODIPine besylate 5 MG Oral Tablet\n\n### Imaging and Procedures:\nShe has undergone several imaging studies relevant to her conditions, including digital radiography of the ankle, wrist, clavicle, and upper arm. Additionally, she has had a coronary artery stent implanted, which is currently active.\n\n### Care Management:\nMrs. Heller is actively involved in various care plans tailored to manage her hypertension and breast cancer, including lifestyle education and routine antenatal care. Her care team at Morton Hospital has been actively managing her treatment and follow-up care.\n\n### Prognosis:\nGiven her age and the active management of her comorbid conditions alongside her breast cancer treatment, the focus remains on maintaining quality of life and managing symptoms. Continuous monitoring and adjustments to her treatment plan are essential to address any progression of her conditions or adverse effects from her medications.\n\nIn summary, Mrs. Heller's health management is complex due to her age, multiple chronic conditions, and breast cancer diagnosis. Her healthcare team is focused on a comprehensive and multidisciplinary approach to provide the best possible outcomes."

         NCT ID  Acronym Overall Status  Start Date  \
1   NCT05483491  Unknown     RECRUITING  2022-09-26   
2   NCT06378294    MACMA     RECRUITING  2021-04-20   
4   NCT03390894  Unknown     RECRUITING  2018-01-18   
12  NCT06135194  Unknown     RECRUITING  2021-04-01   
15  NCT05766891  Unknown     RECRUITING  2023-10-12   

                                           Conditions  \
1   Gastric Cancer, Breast Cancer, Cervical Cancer...   
2                                       Breast Cancer   
4                 Invasive Breast Cancer Early Stages   
12                                      Breast Cancer   
15                                      Breast Cancer   

                       Interventions  \
1   KK-LC-1 TCR-T cells, Aldesleukin   
2                         Mastectomy   
4            No interventions listed   
12                             T-DXd   
15         Group 1, Group 2, Group 3   

                                            Locations Primary Completion Date  \
1   

relatedness = 0.817


'The trial with NCT ID number NCT05044689 has the following eligbility criteria, for inclusion and exclusion Case Group - Patients must have histologically confirmed diagnosis of colorectal or breast cancer.\n\nControl Group - Patients must not have a diagnosis of cancer.'

relatedness = 0.816


'The trial with NCT ID number NCT05406531 has the following eligbility criteria, for inclusion and exclusion Inclusion Criteria:\n\n* Women over the age of 18\n* Breast cancer patients at the end of adjuvant treatment\n\nExclusion Criteria:\n\n* Patients without a mobil device ("smart" mobile phone)\n* Patients without internet access\n* Patients with diagnosed: psychotic disorder; severe depressive episode; panic disorder; bipolar disorder; personality disorder associated with uncontrollable mood swings; PTSD; suicidal attempts and active suicidal ideation; disability pension due to non-specific mental or somatic problems; hospitalization due to a psychiatric disorder in the past year'

relatedness = 0.816


'The trial with NCT ID number NCT05704842 has the following eligbility criteria, for inclusion and exclusion Inclusion Criteria:\n\n* Women with curative-intent breast cancer who plan to undergo at least 4 cycles of chemotherapy.\n* Age \\> 18 years\n* ECOG performance score \\< 3\n* English-speaking\n* with sufficient vision/hearing or family support\n* Coronary artery disease, if cleared by cardiologist\n* Subject must have smart phone, computer or tablet.\n* Willingness to be randomized\n\nExclusion Criteria:\n\n* Medical or psychiatric conditions (beyond those related to breast cancer and its treatment) that would impair our ability to test study hypotheses (psychotic disorders, dementia, inability to give informed consent or follow instructions).\n* Patients with overt evidence of a psychiatric disorder.\n* Coronary artery disease, not cleared by cardiologist.\n* Contraindication to exercise.\n* Chronic fatigue syndrome.'

In [ ]:
print(resp_0)

Based on the provided information about Mrs. Allegra202 Heller342, there are no clinical trials from the provided list that she would be eligible for. Here are the reasons for her ineligibility:

1. **Age and Comorbidities**: Mrs. Heller is 96 years old, which exceeds the typical upper age limit for clinical trial inclusion, which is often capped at 75 or 80 years old in many trials. Additionally, her multiple active comorbid conditions such as essential hypertension, hyperlipidemia, osteoporosis, history of myocardial infarction, and coronary heart disease make her a less suitable candidate for clinical trials that often exclude patients with significant cardiovascular issues or multiple serious health conditions.

2. **Current Medications and Treatments**: Mrs. Heller is on a regimen that includes Tamoxifen and Verzenio for breast cancer treatment, alongside medications for her other health conditions. Many clinical trials exclude patients who are already receiving certain types of c